In [6]:
alpha_name = "AlphaNet_8Input_with_mavg"
shift = 1
sequence = 20
target = "10dRet"
LR = [0.001,0.00001,0.0000001]
epoch_num = [30,30,30]
universe = "Investable"
output_path = "/home/wuwenjun/Data/"

In [7]:
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()

alpha_list = ["Mavg_%s_10" % i for i in ['div_open_close','div_high_close','div_low_close','close','div_vwap_close','volume_sqrt','pct_chg','turnover']]
alpha_list

['Mavg_div_open_close_10',
 'Mavg_div_high_close_10',
 'Mavg_div_low_close_10',
 'Mavg_close_10',
 'Mavg_div_vwap_close_10',
 'Mavg_volume_sqrt_10',
 'Mavg_pct_chg_10',
 'Mavg_turnover_10']

In [8]:
base_list = task_info["Task"].loc[194,"description"]["alpha_list"]
base_list

['COV_div_open_close_div_high_close_10',
 'COV_div_open_close_div_low_close_10',
 'COV_div_open_close_close_10',
 'COV_div_open_close_volume_sqrt_10',
 'COV_div_open_close_pct_chg_10',
 'COV_div_high_close_div_low_close_10',
 'COV_div_high_close_close_10',
 'COV_div_high_close_volume_sqrt_10',
 'COV_div_high_close_pct_chg_10',
 'COV_div_low_close_close_10',
 'COV_div_low_close_volume_sqrt_10',
 'COV_close_div_vwap_close_10',
 'COV_close_volume_sqrt_10',
 'COV_close_pct_chg_10',
 'COV_div_vwap_close_volume_sqrt_10',
 'COV_volume_sqrt_pct_chg_10',
 'COV_volume_sqrt_turnover_10',
 'CORR_div_open_close_div_high_close_10',
 'CORR_div_open_close_div_low_close_10',
 'CORR_div_open_close_close_10',
 'CORR_div_open_close_div_vwap_close_10',
 'CORR_div_open_close_pct_chg_10',
 'CORR_div_high_close_div_low_close_10',
 'CORR_div_high_close_close_10',
 'CORR_div_high_close_div_vwap_close_10',
 'CORR_div_high_close_pct_chg_10',
 'CORR_div_low_close_close_10',
 'CORR_div_low_close_div_vwap_close_10',

In [9]:
import time
import datetime
import os
from Research.backtest.bt import BTDaily
import matplotlib.pyplot as plt
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
import AlphaNet
from AlphaNet.Data import concat_original_data , generate_shift_data

In [ ]:
if alpha_name not in os.listdir(output_path):
    _ , alpha_list = concat_original_data(alpha_name=alpha_name,alpha_list=alpha_list,output_path=output_path,universe = universe,corr_filter = 0.8,base_list = base_list)
generate_shift_data(alpha_name=alpha_name,shift=shift,sequence=sequence,target=target,data_path=output_path)

# Task Generation
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()
original = task_info["Task"]
# Timelist & Task
time_list = ["2019-01-01","2019-06-01","2020-01-01","2020-06-01","2021-01-01","2021-06-01"]
time_list2 = []
for i in range(len(time_list)-1):
    time_list2.append([time_list[i],time_list[i+1]])
time_list2 = pd.DataFrame(time_list2,columns=["start_date","end_date"])

task = pd.DataFrame([
                    alpha_name + "_Shift_%i_Sequence_%i_%s" % (shift,sequence,target),
                    sequence,LR,epoch_num,len(alpha_list),
                        ],index=["Alpha_Name","sequence","LR","epoch_num","feature_num"]).T
task['value']=1
time_list2['value']=1
task = pd.merge(task,time_list2,how='left',on='value')
del task['value']
task["status"] = "waiting"
task["description"] = [{
    "target" : target,
    "alpha_name" : alpha_name,
    "alpha_list" : alpha_list,
    "shift" : shift,
    "sequence" : sequence
} for i in range(len(task))]
task = pd.concat([original,task],axis=0)
task.reset_index(drop=True,inplace=True)
task.index.names = ["task_id"]
task_info["Task"] = task
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/Task.npy",task_info)

Calculating correlations...: 100%|███████████████████████████████████████████████████████████████████████████████| 1306/1306 [00:05<00:00, 226.32it/s]


delete [Mavg_div_open_close_10] because of 93 [ZSCORE_div_open_close_10]


Calculating correlations...: 100%|███████████████████████████████████████████████████████████████████████████████| 1306/1306 [00:05<00:00, 237.46it/s]


delete [Mavg_div_high_close_10] because of 94 [DECAY_div_high_close_10]


Calculating correlations...: 100%|███████████████████████████████████████████████████████████████████████████████| 1306/1306 [00:05<00:00, 222.50it/s]


...

delete [Mavg_volume_sqrt_10] because of 84 [STD_volume_sqrt_10]


Calculating correlations...: 100%|███████████████████████████████████████████████████████████████████████████████| 1306/1306 [00:06<00:00, 216.44it/s]


delete [Mavg_pct_chg_10] because of 92 [ZSCORE_pct_chg_10]


Calculating correlations...: 100%|███████████████████████████████████████████████████████████████████████████████| 1306/1306 [00:05<00:00, 218.74it/s]


delete [Mavg_turnover_10] because of 98 [DECAY_turnover_10]
